In [41]:
import numpy as np
import pandas as pd
from functools import reduce
import re

## 准备：从文本制作词向量

以词向量或标记向量的形式查看文本，即将句子转换为向量。

将每个单独的文档转换为词汇表中的向量

In [25]:
def loadDataSet():
    postingList = [
        ['my', 'dog', 'has', 'flea', 'problems', 'help', 'please'],
        ['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid'],
        ['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him'],
        ['stop', 'posting', 'stupid', 'worthless', 'garbage'],
        ['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him'],
        ['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']
    ]
    classVec = [0, 1, 0, 1, 0, 1]  #1 is abusive, 0 not
    return postingList, classVec

- 第一个函数创建一些示例数据进行实验。从 loadDatSet() 返回的第一个变量是来自达尔马提亚（斑点狗）爱好者留言板的一组标记化文档。文本已被分解为一组标记。标点符号也已从该文本中删除。
- loadDatSet() 的第二个变量返回一组类标签。这里有两个类别，辱骂类和非辱骂类。
- postingList - 实验样本切分的词条
- classVec - 类别标签向量

In [26]:
def createVocabList(dataSet):
    vocabSet = set([])  #create empty set
    for document in dataSet:
        vocabSet = vocabSet | set(document)  #union of the two sets
    return list(vocabSet)

- 函数 createVocabList() 将创建所有文档中所有唯一单词的列表。首先，创建一个空集。接下来，将每个文档中的新集合附加到该集合中。
- dataSet - 整理的样本数据集
- vocabSet - 返回不重复的词条列表，也就是词汇表

In [27]:
def setOfWords2Vec(vocabList, inputSet):
    returnVec = [0] * len(vocabList)
    for word in inputSet:
        if word in vocabList:
            returnVec[vocabList.index(word)] = 1
        else:
            print("the word: %s is not in my Vocabulary!" % word)
    return returnVec

- vocabList - createVocabList返回的列表
- inputSet - 切分的词条列表
- returnVec - 文档向量,词集模型
- 最后，在获得词汇列表后，使用函数 setOfWords2Vec()，该函数获取词汇列表和文档，并输出由 1 和 0 组成的向量来表示词汇表中的单词是否存在于给定文档中。然后创建一个与词汇列表长度相同的向量并用 0 填充它。 
- 接下来浏览文档中的单词，如果该单词在词汇列表中，则在输出向量中将其值设置为 1。如果一切顺利，不需要测试某个单词是否在 vocabList 中，但稍后可能会使用它。

**函数实际效果**

In [28]:
postingList, classVec = loadDataSet()
for each in postingList:
    print(each)
print(classVec)

['my', 'dog', 'has', 'flea', 'problems', 'help', 'please']
['maybe', 'not', 'take', 'him', 'to', 'dog', 'park', 'stupid']
['my', 'dalmation', 'is', 'so', 'cute', 'I', 'love', 'him']
['stop', 'posting', 'stupid', 'worthless', 'garbage']
['mr', 'licks', 'ate', 'my', 'steak', 'how', 'to', 'stop', 'him']
['quit', 'buying', 'worthless', 'dog', 'food', 'stupid']
[0, 1, 0, 1, 0, 1]


- postingList是存放词条列表中，classVec是存放每个词条的所属类别，1代表侮辱类 ，0代表非侮辱类。

## 训练：根据词向量计算概率

得到词向量后，通过词向量训练朴素贝叶斯分类器。

In [29]:
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)  #计算训练的文档数目
    numWords = len(trainMatrix[0])  #计算每篇文档的词条数
    pAbusive = sum(trainCategory) / float(numTrainDocs)  #文档属于侮辱类的概率
    p0Num = np.zeros(numWords)
    p1Num = np.zeros(numWords)  #创建numpy.zeros数组,词条出现数初始化为0
    p0Denom = 0.0
    p1Denom = 0.0  #分母初始化为0
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:  #统计属于侮辱类的条件概率所需的数据，即P(w0|1),P(w1|1),P(w2|1)···
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:  #统计属于非侮辱类的条件概率所需的数据，即P(w0|0),P(w1|0),P(w2|0)···
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = p1Num / p1Denom
    p0Vect = p0Num / p0Denom
    return p0Vect, p1Vect, pAbusive  #返回属于侮辱类的条件概率数组，属于非侮辱类的条件概率数组，文档属于侮辱类的概率

- trainMatrix - 训练文档矩阵，即setOfWords2Vec返回的returnVec构成的矩阵
- trainCategory - 训练类别标签向量，即loadDataSet返回的classVec
- p0Vect - 侮辱类的条件概率数组
- p1Vect - 非侮辱类的条件概率数组
- pAbusive - 文档属于侮辱类的概率

In [30]:
postingList, classVec = loadDataSet()
myVocabList = createVocabList(postingList)
print('myVocabList:\n', myVocabList)
trainMat = []
for postinDoc in postingList:
    trainMat.append(setOfWords2Vec(myVocabList, postinDoc))
p0V, p1V, pAb = trainNB0(trainMat, classVec)
print('p0V:\n', p0V)
print('p1V:\n', p1V)
print('classVec:\n', classVec)
print('pAb:\n', pAb)

myVocabList:
 ['take', 'to', 'ate', 'please', 'worthless', 'stop', 'flea', 'maybe', 'cute', 'has', 'dog', 'stupid', 'steak', 'quit', 'buying', 'not', 'so', 'posting', 'how', 'food', 'mr', 'garbage', 'park', 'licks', 'I', 'dalmation', 'is', 'him', 'my', 'problems', 'love', 'help']
p0V:
 [0.         0.04166667 0.04166667 0.04166667 0.         0.04166667
 0.04166667 0.         0.04166667 0.04166667 0.04166667 0.
 0.04166667 0.         0.         0.         0.04166667 0.
 0.04166667 0.         0.04166667 0.         0.         0.04166667
 0.04166667 0.04166667 0.04166667 0.08333333 0.125      0.04166667
 0.04166667 0.04166667]
p1V:
 [0.05263158 0.05263158 0.         0.         0.10526316 0.05263158
 0.         0.05263158 0.         0.         0.10526316 0.15789474
 0.         0.05263158 0.05263158 0.05263158 0.         0.05263158
 0.         0.05263158 0.         0.05263158 0.05263158 0.
 0.         0.         0.         0.05263158 0.         0.
 0.         0.        ]
classVec:
 [0, 1, 0, 

- p0V存放的是每个单词属于类别0，也就是非侮辱类词汇的概率。比如p0V的倒数第6个概率，就是stupid这个单词属于非侮辱类的概率为0。同理，p1V的倒数第6个概率，就是stupid这个单词属于侮辱类的概率为0.15789474，也就是约等于15.79%的概率。
- stupid的中文意思是蠢货，显而易见，这个单词属于侮辱类。
- pAb是所有侮辱类的样本占所有样本的概率，从classVec中可以看出，一用有3个侮辱类，3个非侮辱类。所以侮辱类的概率是0.5。因此p0V存放的就是P(him|非侮辱类) = 0.0833、P(is|非侮辱类) = 0.0417，一直到P(dog|非侮辱类) = 0.0417，这些单词的条件概率。同理，p1V存放的就是各个单词属于侮辱类的条件概率。pAb就是先验概率。

## 测试：根据现实条件修改分类器

In [31]:
def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = reduce(lambda x, y: x * y, vec2Classify * p1Vec) * pClass1  #对应元素相乘
    p0 = reduce(lambda x, y: x * y, vec2Classify * p0Vec) * (1.0 - pClass1)
    print('p0:', p0)
    print('p1:', p1)
    if p1 > p0:
        return 1
    else:
        return 0

- vec2Classify - 待分类的词条数组
- p0Vec - 非侮辱类的条件概率数组
- p1Vec -侮辱类的条件概率数组
- pClass1 - 文档属于侮辱类的概率
- returns: 0 - 属于非侮辱类; 1 - 属于侮辱类

In [32]:
def testingNB():
    listOPosts, listClasses = loadDataSet()  #创建实验样本
    myVocabList = createVocabList(listOPosts)  #创建词汇表
    trainMat = []
    for postinDoc in listOPosts:
        trainMat.append(setOfWords2Vec(myVocabList, postinDoc))  #将实验样本向量化
    p0V, p1V, pAb = trainNB0(np.array(trainMat),
                             np.array(listClasses))  #训练朴素贝叶斯分类器
    testEntry = ['love', 'my', 'dalmation']  #测试样本1
    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))  #测试样本向量化
    if classifyNB(thisDoc, p0V, p1V, pAb):
        print(testEntry, '属于侮辱类')  #执行分类并打印分类结果
    else:
        print(testEntry, '属于非侮辱类')  #执行分类并打印分类结果
    testEntry = ['stupid', 'garbage']  #测试样本2

    thisDoc = np.array(setOfWords2Vec(myVocabList, testEntry))  #测试样本向量化
    if classifyNB(thisDoc, p0V, p1V, pAb):
        print(testEntry, '属于侮辱类')  #执行分类并打印分类结果
    else:
        print(testEntry, '属于非侮辱类')

In [33]:
testingNB()

p0: 0.0
p1: 0.0
['love', 'my', 'dalmation'] 属于非侮辱类
p0: 0.0
p1: 0.0
['stupid', 'garbage'] 属于非侮辱类


- 创建了两个测试样本：['love', 'my', 'dalmation']和['stupid', 'garbage']

**Problems：**
- 发现无法正确分类，原因：利用贝叶斯分类器对文档进行分类时，要计算多个概率的乘积以获得文档属于某个类别的概率，即计算p(w0|1)p(w1|1)p(w2|1)。如果其中有一个概率值为0，那么最后的成绩也为0。
- 除此之外，另外一个遇到的问题就是**下溢出**，这是由于太多很小的数相乘造成的。为了解决这个问题，对乘积结果取**自然对数**。通过求对数可以避免下溢出或者浮点数舍入导致的错误。同时，采用自然对数进行处理不会有任何损失。

修改test函数和classifyNB函数：

In [34]:
def trainNB0(trainMatrix, trainCategory):
    numTrainDocs = len(trainMatrix)  #计算训练的文档数目
    numWords = len(trainMatrix[0])  #计算每篇文档的词条数
    pAbusive = sum(trainCategory) / float(numTrainDocs)  #文档属于侮辱类的概率
    p0Num = np.ones(numWords)
    p1Num = np.ones(numWords)  #创建numpy.ones数组,词条出现数初始化为1，拉普拉斯平滑
    p0Denom = 2.0
    p1Denom = 2.0  #分母初始化为2,拉普拉斯平滑
    for i in range(numTrainDocs):
        if trainCategory[i] == 1:  #统计属于侮辱类的条件概率所需的数据，即P(w0|1),P(w1|1),P(w2|1)···
            p1Num += trainMatrix[i]
            p1Denom += sum(trainMatrix[i])
        else:  #统计属于非侮辱类的条件概率所需的数据，即P(w0|0),P(w1|0),P(w2|0)···
            p0Num += trainMatrix[i]
            p0Denom += sum(trainMatrix[i])
    p1Vect = np.log(p1Num / p1Denom)  #取对数，防止下溢出
    p0Vect = np.log(p0Num / p0Denom)
    return p0Vect, p1Vect, pAbusive


def classifyNB(vec2Classify, p0Vec, p1Vec, pClass1):
    p1 = sum(vec2Classify * p1Vec) + np.log(
        pClass1)  #对应元素相乘。logA * B = logA + logB
    p0 = sum(vec2Classify * p0Vec) + np.log(1.0 - pClass1)
    if p1 > p0:
        return 1
    else:
        return 0

In [35]:
testingNB()

['love', 'my', 'dalmation'] 属于非侮辱类
['stupid', 'garbage'] 属于侮辱类


**结果正确！！！**

## 过滤垃圾邮件

In [36]:
# 接收一个 大字符串 并将其解析为 字符串列表
def textParse(bigString):  #将字符串转换为字符列表
    listOfTokens = re.split(r'\W', bigString)  #将特殊符号作为切分标志进行字符串切分，即非字母、非数字
    return [tok.lower() for tok in listOfTokens if len(tok) > 2]  #除了单个字母，例如大写的I，其它单词变成小写


# 将切分的实验样本词条整理成不重复的词条列表，也就是词汇表
def createVocabList(dataSet):
    vocabSet = set([])  #创建一个空的不重复列表
    for document in dataSet:
        vocabSet = vocabSet | set(document)  #取并集
    return list(vocabSet)

- 原书中re.split(r'\W*')匹配任何非字母数字字符的序列，包括没有字符的情况（即空字符串，0长度），因此无法成功运行，这里改为'\W'即可

In [37]:
docList = []
classList = []
for i in range(1, 26):  #遍历25个txt文件
    wordList = textParse(open('./datasets/email/spam/%d.txt' % i,
                              'r').read())  #读取每个垃圾邮件，并字符串转换成字符串列表
    docList.append(wordList)
    classList.append(1)  #标记垃圾邮件，1表示垃圾文件
    wordList = textParse(open('./datasets/email/ham/%d.txt' % i,
                              'r').read())  #读取每个非垃圾邮件，并字符串转换成字符串列表
    docList.append(wordList)
    classList.append(0)  #标记非垃圾邮件，1表示垃圾文件
vocabList = createVocabList(docList)  #创建词汇表，不重复
print(vocabList)

['color', 'email', 'art', 'benoit', 'because', 'plus', 'gains', '90563', 'attaching', 'leaves', 'individual', 'life', 'how', '50mg', 'year', 'way', 'will', 'over', 'inform', 'been', 'moderately', 'pharmacy', 'help', 'sites', 'logged', 'development', 'save', 'mathematician', 'forum', 'received', 'item', 'now', 'yeah', 'ryan', '292', 'tent', 'pictures', 'ideas', 'volume', 'bargains', '5mg', 'saw', 'and', 'doing', 'sophisticated', 'new', 'low', 'with', 'tesla', 'from', 'that', 'roofer', 'watson', 'approved', 'shipping', 'jocelyn', 'them', 'ordercializviagra', 'told', 'competitive', '513', 'release', 'most', 'fans', 'night', 'wilmott', 'see', 'such', 'fedex', 'pick', 'place', 'window', 'hello', 'going', 'went', 'out', 'then', 'inside', 'take', 'windows', 'zolpidem', 'pricing', 'comment', '100m', 'let', 'free', '2007', 'gucci', 'concise', '100', 'www', 'buyviagra', 'cca', 'wholesale', 'reply', 'wilson', 'decision', 'cold', 'strategic', 'assistance', 'have', 'discreet', 'shipment', 'storedet

### Train and Test

将数据集分为训练集和测试集，使用交叉验证的方式测试朴素贝叶斯分类器的准确性

In [38]:
# 测试朴素贝叶斯分类器
def spamTest():
    docList = []
    classList = []
    fullText = []
    for i in range(1, 26):  #遍历25个txt文件
        wordList = textParse(open('./datasets/email/spam/%d.txt' % i,
                                  'r').read())  #读取每个垃圾邮件，并字符串转换成字符串列表
        docList.append(wordList)
        fullText.append(wordList)
        classList.append(1)  #标记垃圾邮件，1表示垃圾文件
        wordList = textParse(open('./datasets/email/ham/%d.txt' % i,
                                  'r').read())  #读取每个非垃圾邮件，并字符串转换成字符串列表
        docList.append(wordList)
        fullText.append(wordList)
        classList.append(0)  #标记非垃圾邮件，1表示垃圾文件
    vocabList = createVocabList(docList)  #创建词汇表，不重复
    trainingSet = list(range(50))
    testSet = []  #创建存储训练集的索引值的列表和测试集的索引值的列表
    for i in range(10):  #从50个邮件中，随机挑选出40个作为训练集,10个做测试集
        randIndex = int(np.random.uniform(0, len(trainingSet)))  #随机选取索索引值
        testSet.append(trainingSet[randIndex])  #添加测试集的索引值
        del (trainingSet[randIndex])  #在训练集列表中删除添加到测试集的索引值
    trainMat = []
    trainClasses = []  #创建训练集矩阵和训练集类别标签系向量
    for docIndex in trainingSet:  #遍历训练集
        trainMat.append(setOfWords2Vec(vocabList,
                                       docList[docIndex]))  #将生成的词集模型添加到训练矩阵中
        trainClasses.append(classList[docIndex])  #将类别添加到训练集类别标签系向量中
    p0V, p1V, pSpam = trainNB0(np.array(trainMat),
                               np.array(trainClasses))  #训练朴素贝叶斯模型
    errorCount = 0  #错误分类计数
    for docIndex in testSet:  #遍历测试集
        wordVector = setOfWords2Vec(vocabList, docList[docIndex])  #测试集的词集模型
        if classifyNB(np.array(wordVector), p0V, p1V,
                      pSpam) != classList[docIndex]:  #如果分类错误
            errorCount += 1  #错误计数加1
            print("分类错误的测试集：", docList[docIndex])
    print('错误率：%.2f%%' % (float(errorCount) / len(testSet) * 100))

In [39]:
spamTest()

错误率：0.00%


**效果不错！！！**